# Esquema de firmas SDitH

Finalmente, construimos una versión simplificada del esquema de firmas SDitH. Este esquema de firmas usa la prueba de cero conocimiento no interactiva para el protocolo MPC SDitH, la cual hace uso de la heurística de Fiat-Shamir. A partir de este protocolo no interactivo, construimos el esquema de firmas.

La secuencia de pasos que vamos a seguir para el algoritmo de firma es la siguiente: 

1. Generación de Claves:
   - Se genera un par de claves: una clave privada y una clave pública.
   - La clave privada se utiliza para firmar los mensajes, mientras que la clave pública se usa para verificar las firmas.

2. Firma del Mensaje:
   - Dado un mensaje, el firmante utiliza la clave privada para crear una firma.
   - La firma debe garantizar que cualquier alteración del mensaje pueda ser detectada.

3. Verificación de la Firma:
   - El verificador utiliza la clave pública junto con la firma y el mensaje para verificar su validez.
   - Si la verificación es exitosa, se garantiza la autenticidad e integridad del mensaje; si falla, se rechaza la firma.

Este esquema asegura que el mensaje proviene de la parte que posee la clave privada y que no ha sido alterado, garantizando tanto la autenticidad como la integridad del mensaje.


## Generación de Claves

En este paso, se realiza la generación de las claves necesarias para el esquema de firma basado en el problema de decodificación del síndrome.

- Clave pública: Consiste en una matriz H y un síndrome y. La matriz $H$ se genera en una forma estandarizada, que incluye una matriz identidad y una parte aleatoria. Esta matriz $H$ es utilizada en el proceso de codificación para asegurar que se pueda verificar el resultado sin conocer los valores secretos. El síndrome $y$ se calcula multiplicando la matriz $H$ con un vector secreto $x$.

- Clave privada: Es el vector x, el cual tiene un peso de Hamming específico. Este vector se crea de tal manera que solamente una cantidad limitada de posiciones tiene valores distintos de cero. Este valor limitado asegura la dificultad del problema de decodificación, lo que aporta seguridad al esquema.

La clave pública se comparte para permitir la verificación, mientras que la clave privada se mantiene secreta. Se utiliza una semilla aleatoria para asegurar que la generación sea reproducible, lo cual es crucial para pruebas y validación del esquema.

In [1]:
# Importamos las bibliotecas necesarias
import random
import hashlib

n_parts = 5  # Número de partes

# Definir los campos finitos
q = 2  # Número primo para definir el campo finito
p = 23  # Número primo mayor para definir un campo finito más grande
F_q = GF(q)
F_p = GF(p)
n = 6  # Longitud del vector de error
k = 3  # Longitud del vector de mensaje
m = n - k  # Número de filas de la matriz de paridad H
w = 3

def generar_claves(n=10, k=5, w=3, semilla=42):
    # Paso 1: Definir los parámetros del problema
    # n: Número de columnas (longitud del vector x)
    # k: Número de filas de la matriz H
    # w: Peso de Hamming del vector x

    # Establecer la semilla para la generación reproducible
    set_random_seed(semilla)  # Semilla para Sage

    # Paso 2: Generar la matriz H en forma estándar con la identidad a la derecha
    matriz_aleatoria = Matrix(F_q, k, n - k, lambda i, j: F_q.random_element())
    identidad = Matrix.identity(F_q, k)
    H = matriz_aleatoria.augment(identidad)

    # Paso 3: Generar el vector x con peso de Hamming w
    x = vector(F_q, n)  # Inicializar el vector x con ceros
    indices_no_nulos = sample(range(n), w)  # Seleccionar aleatoriamente w índices distintos

    for idx in indices_no_nulos:
        x[idx] = F_q.random_element(primitive=True)  # Asignar un valor no nulo en las posiciones seleccionadas

    # Paso 4: Calcular el vector y (síndrome)
    y = H * x

    # Paso 5: Salida
    clave_publica = (H, y)
    clave_privada = (H, x, y)
    return clave_publica, clave_privada

# Llamada a la función para generar las claves
clave_publica, clave_privada = generar_claves(n,k,w, 1)

print("\nClave privada (Matriz H, Vector x, Vector y):")
print("\nMatriz H:")
print(clave_privada[0])
print("\nVector x:")
print(clave_privada[1])
print("Vector y (síndrome):")
print(clave_privada[2])

print("\nClave pública (Matriz H, Vector y):")
print("\nMatriz H:")
print(clave_publica[0])
print("\nVector y (síndrome):")
print(clave_publica[1])


Clave privada (Matriz H, Vector x, Vector y):

Matriz H:
[1 0 1 1 0 0]
[1 0 1 0 1 0]
[0 0 0 0 0 1]

Vector x:
(0, 0, 1, 0, 1, 0)
Vector y (síndrome):
(1, 0, 0)

Clave pública (Matriz H, Vector y):

Matriz H:
[1 0 1 1 0 0]
[1 0 1 0 1 0]
[0 0 0 0 0 1]

Vector y (síndrome):
(1, 0, 0)


### Firma del mensaje
Establecemos el mensaje de interés a firmar.

In [2]:
mensaje = "Este es el mensaje a firmar"

(0, 0, 1, 0, 1, 0)

### Preparación del protocolo MPC

En este paso, preparamos los elementos necesarios para la ejecución del protocolo de omputación ultipartita, que es la base de nuestro esquema de firmas. La preparación incluye lo siguiente:

- Configuración del campo finito: El protocolo MPC se realiza sobre un campo finito, que define el conjunto de valores posibles y asegura que las operaciones sean bien definidas y eficientes. En esta etapa, definimos todos los parámetros necesarios para ejecutar el protocolo MPC.

- Compartición de los valores secretos: A partir del valor secreto $x$ se crean los polinomios asociados a este y se comparten entre diferentes partes usando un esquema de compartición de secretos. Esto permite que ninguna parte individual tenga acceso completo al secreto, asegurando la privacidad del valor.

- Generación de compromisos: Para cada valor compartido, se genera un compromiso. Estos compromisos actúan como un "cierre seguro" sobre las comparticiones para asegurar que los valores revelados más adelante sean consistentes con los originales. La generación de compromisos se realiza utilizando una función hash sobre cada compartición, lo que garantiza la integridad de los valores.

Este proceso es fundamental para establecer la base de una prueba segura que permita demostrar conocimiento sin revelar información confidencial. Al compartir el valor secreto y generar compromisos, nos aseguramos de que las partes puedan trabajar en conjunto sin que ninguna de ellas tenga acceso al secreto completo.


In [3]:
# Definir el vector x en el campo F_q de longitud n
x = clave_privada[1]  # Ejemplo con valores específicos

# Asociar los primeros n elementos del campo F_p con las coordenadas de x
f = [F_p(i) for i in range(n)]  # f_i para i en [1:n]

# Polinomio S: Interpolación de Lagrange usando los puntos (f_i, x_i)
R = F_p['X']
S = R.lagrange_polynomial([(f[i], x[i]) for i in range(n)])

# Polinomio Q: Producto de (X - f_i) para i en el subconjunto de índices con valor x_i = 1
X = R.gen()
E = [i for i in range(n) if x[i] != 0]  # Subconjunto de índices no nulos de x
Q = prod([(X - f[i]) for i in E])

# Polinomio F: Polinomio de "desaparición" para todos los f_i
F = prod([(X - f[i]) for i in range(n)])

# Polinomio P: Definido como P = S * Q / F
P = (S * Q).quo_rem(F)[0]  # División polinomial asegurando que F divide a S * Q sin residuo

# Evaluar los polinomios S y Q en los puntos f
evaluaciones_S = [S(f_i) for f_i in f]
evaluaciones_Q = [Q(f_i) for f_i in f]

# Imprimir los polinomios
def imprimir_polinomios():
    print("Polinomio S (Interpolación de Lagrange):")
    print(S)
    print("\nEvaluaciones de S en f:")
    print(evaluaciones_S)
    print("\nSubconjunto E:")
    print(E)
    print("\nPolinomio Q (Producto de raíces para índices no nulos de x):")
    print(Q)
    print("\nEvaluaciones de Q en f:")
    print(evaluaciones_Q)
    print("\nPolinomio F (Vanishing Polynomial):")
    print(F)
    print("\nPolinomio P (Resultado de S * Q / F):")
    print(P)
    
    print("\nEvaluación de Polinomios")
    print(f"\nS*Q==P*F: {S*Q==P*F}")

imprimir_polinomios()

Polinomio S (Interpolación de Lagrange):
20*X^5 + 14*X^4 + 2*X^3 + 22*X^2 + 11*X

Evaluaciones de S en f:
[0, 0, 1, 0, 1, 0]

Subconjunto E:
[2, 4]

Polinomio Q (Producto de raíces para índices no nulos de x):
X^2 + 17*X + 8

Evaluaciones de Q en f:
[8, 3, 0, 22, 0, 3]

Polinomio F (Vanishing Polynomial):
X^6 + 8*X^5 + 16*X^4 + 5*X^3 + 21*X^2 + 18*X

Polinomio P (Resultado de S * Q / F):
20*X + 10

Evaluación de Polinomios

S*Q==P*F: True


In [4]:
def dividir_polinomio_en_partes(polinomio, n_parts, campo):
    coeficientes = polinomio.coefficients(sparse=False)
    grado = polinomio.degree()
    partes = [[] for _ in range(n_parts)]  # Crear listas para cada parte

    for i in range(grado + 1):
        # Generamos aleatoriamente las primeras (n_parts - 1) partes en el campo
        partial_shares = [campo.random_element() for _ in range(n_parts - 1)]
        # Calculamos la última parte de manera que la suma de todas las partes sea igual al coeficiente original
        last_share = coeficientes[i] - sum(partial_shares)
        # Añadimos la última parte a la lista de partes
        partial_shares.append(last_share)
        # Distribuimos cada parte correspondiente entre las listas de partes de los participantes
        for j in range(n_parts):
            partes[j].append(partial_shares[j])
        
        # Imprimir las partes generadas para el coeficiente actual
        print(f"Coeficiente de x^{i} ({coeficientes[i]}): Partes -> {partial_shares}")

    return partes

# Función para generar los polinomios a partir de las partes
def construir_polinomios_de_partes(shares, n_parts, X):
    polinomios = []
    for j in range(n_parts):
        part_poly = sum(shares[j][i] * X^i for i in range(len(shares[j])))
        polinomios.append(part_poly)
    return polinomios

# Dividimos los polinomios S, Q y P en partes
print("\nDividiendo el polinomio S en partes:")
shares_S = dividir_polinomio_en_partes(S, n_parts, F_p)

print("\nDividiendo el polinomio Q en partes:")
shares_Q = dividir_polinomio_en_partes(Q, n_parts, F_p)

print("\nDividiendo el polinomio P en partes:")
shares_P = dividir_polinomio_en_partes(P, n_parts, F_p)

# Construimos los polinomios para cada una de las partes
def imprimir_polinomios_partes():
    partes_S_polinomios = construir_polinomios_de_partes(shares_S, n_parts, X)
    partes_Q_polinomios = construir_polinomios_de_partes(shares_Q, n_parts, X)
    partes_P_polinomios = construir_polinomios_de_partes(shares_P, n_parts, X)

    print("\nPolinomio que recibe cada parte para el polinomio S:")
    for j, part_poly in enumerate(partes_S_polinomios):
        print(f"Parte {j + 1} de S: {part_poly}")

    print("\nPolinomio que recibe cada parte para el polinomio Q:")
    for j, part_poly in enumerate(partes_Q_polinomios):
        print(f"Parte {j + 1} de Q: {part_poly}")

    print("\nPolinomio que recibe cada parte para el polinomio P:")
    for j, part_poly in enumerate(partes_P_polinomios):
        print(f"Parte {j + 1} de P: {part_poly}")

imprimir_polinomios_partes()

# Función para generar valores aleatorios en el campo
def generar_valores_aleatorios(campo, n_parts):
    valores = [campo.random_element() for _ in range(n_parts)]
    print(f"Valores aleatorios generados: {valores}")
    return valores

# Generar valores aleatorios para a_shares y b_shares
print("\nGenerando partes para a y b:")
a_shares = generar_valores_aleatorios(F_p, n_parts)
b_shares = generar_valores_aleatorios(F_p, n_parts)

# Calcular el producto c = a * b, y luego compartir el valor c
c = sum(a_shares) * sum(b_shares)
print(f"\nValor de c (producto de a y b): {c}")

# Generar partes para c_shares
c_shares = generar_valores_aleatorios(F_p, n_parts)
c_sum = sum(c_shares)
adjustment = c - c_sum
print(f"\nSuma de las partes de c antes del ajuste: {c_sum}")
print(f"Ajuste necesario para c: {adjustment}")

# Ajustar la primera parte de c_shares para garantizar que sum(c_shares) == c
c_shares[0] += adjustment
print(f"Partes ajustadas de c: {c_shares}")

print("\nResumen de lo que recibe cada parte:")
for i in range(n_parts):
    print(f"Parte {i + 1} recibe:")
    print(f"  - Coeficientes de S: {shares_S[i]}")
    print(f"  - Coeficientes de Q: {shares_Q[i]}")
    print(f"  - Coeficientes de P: {shares_P[i]}")
    print(f"  - Parte de a: {a_shares[i]}")
    print(f"  - Parte de b: {b_shares[i]}")
    print(f"  - Parte de c: {c_shares[i]}")


Dividiendo el polinomio S en partes:
Coeficiente de x^0 (0): Partes -> [9, 17, 11, 12, 20]
Coeficiente de x^1 (11): Partes -> [18, 22, 7, 21, 12]
Coeficiente de x^2 (22): Partes -> [20, 9, 1, 6, 9]
Coeficiente de x^3 (2): Partes -> [0, 10, 13, 20, 5]
Coeficiente de x^4 (14): Partes -> [22, 13, 17, 21, 10]
Coeficiente de x^5 (20): Partes -> [10, 21, 5, 1, 6]

Dividiendo el polinomio Q en partes:
Coeficiente de x^0 (8): Partes -> [14, 20, 3, 5, 12]
Coeficiente de x^1 (17): Partes -> [2, 5, 21, 2, 10]
Coeficiente de x^2 (1): Partes -> [15, 16, 4, 8, 4]

Dividiendo el polinomio P en partes:
Coeficiente de x^0 (10): Partes -> [12, 6, 4, 19, 15]
Coeficiente de x^1 (20): Partes -> [18, 1, 9, 14, 1]

Polinomio que recibe cada parte para el polinomio S:
Parte 1 de S: 10*X^5 + 22*X^4 + 20*X^2 + 18*X + 9
Parte 2 de S: 21*X^5 + 13*X^4 + 10*X^3 + 9*X^2 + 22*X + 17
Parte 3 de S: 5*X^5 + 17*X^4 + 13*X^3 + X^2 + 7*X + 11
Parte 4 de S: X^5 + 21*X^4 + 20*X^3 + 6*X^2 + 21*X + 12
Parte 5 de S: 6*X^5 + 10

In [5]:
# Función para obtener el commit de cada parte usando una función hash
def obtener_commit_de_parte_hash(shares_S, shares_Q, shares_P, a_shares, b_shares, c_shares, campo):
    commits = []
    valores_aleatorios = []
    for i in range(n_parts):
        # Generar un valor aleatorio para cada parte
        valor_aleatorio = campo.random_element()
        valores_aleatorios.append(valor_aleatorio)
        
        # Crear una cadena con todos los valores para la parte i
        datos_str = (
            f"{shares_S[i]}{shares_Q[i]}{shares_P[i]}"
            f"{a_shares[i]}{b_shares[i]}{c_shares[i]}"
            f"{valor_aleatorio}"
        )
        
        # Calcular el hash usando SHA-256
        commit = hashlib.sha256(datos_str.encode()).hexdigest()
        commits.append(commit)
        print(f"Commit de la parte {i + 1}: {commit} (valor aleatorio utilizado: {valor_aleatorio})")

    return commits, valores_aleatorios

# Obtener los commits de cada parte usando la función hash
print("\nObteniendo los commits de cada parte usando hash:")
commits, valores_aleatorios = obtener_commit_de_parte_hash(shares_S, shares_Q, shares_P, a_shares, b_shares, c_shares, F_p)


Obteniendo los commits de cada parte usando hash:
Commit de la parte 1: fe5aee9ed25f6ecaf1d1275d59a77904bdd9fade7c66eefbc5b62cd502e9e1f6 (valor aleatorio utilizado: 0)
Commit de la parte 2: 120f694616028fe9c6966faf3168ee9120a22c800d01162fb673478f01ae0014 (valor aleatorio utilizado: 4)
Commit de la parte 3: 12ab9e4054b4562477a9c593f64a1dc1ddaa9ef897e469436f30f5597258bad4 (valor aleatorio utilizado: 12)
Commit de la parte 4: 7977ac33042c57e71e631eaad7ad6b653a7950501d82e66c31f19f41cd6667fa (valor aleatorio utilizado: 21)
Commit de la parte 5: 65315fffdc8313ed82df23e13ba072f2c15e07f8becd04cbc5e137c1daba5b4b (valor aleatorio utilizado: 0)


In [6]:
# Paso 2: Derivación del primer desafío aplicando la función hash a los compromisos

# Concatenar todos los commits para formar una única cadena
commits_concatenados = mensaje + ''.join(str(commit) for commit in commits)

# Calcular h1 aplicando SHA-256 a la cadena de commits concatenados
h1_hex = hashlib.sha256(commits_concatenados.encode()).hexdigest()
print(f"\nValor de h1 (hash de los compromisos concatenados): {h1_hex}")

# Convertir h1 de hexadecimal a un valor numérico para usarlo como semilla
h1_int = int(h1_hex, 16)

# Inicializar el generador aleatorio con la semilla h1
random.seed(h1_int)

# Convertir F_p a un entero para trabajar con random
F_p_int = int(F_p.order())  # Obtiene el orden del campo finito como un entero

# Derivar r y epsilon a partir del generador aleatorio
r = random.randint(1, F_p_int - 1)  # Generar un valor aleatorio r en el rango del campo finito
epsilon = random.randint(1, F_p_int - 1)  # Generar un valor aleatorio epsilon en el rango del campo finito

# Convertir r y epsilon a elementos del campo finito F_p
r = F_p(r)
epsilon = F_p(epsilon)

print(f"\nValores derivados de h1:")
print(f"r: {r}")
print(f"epsilon: {epsilon}")


Valor de h1 (hash de los compromisos concatenados): f8b4c73af60f109f90bea2428462694e62cf37debb3db80a127bb7c95d34a7d5

Valores derivados de h1:
r: 22
epsilon: 15


In [7]:
# Implementación del protocolo MPC

# Paso 2: Evaluar los polinomios S, Q, y P en un punto aleatorio r del campo finito
def evaluar_polinomios_en_punto(polinomios, r):
    return [part(r) for part in polinomios]

print(f"Valor aleatorio {r}")
print(f"Valuación en punto aleatorio {S(r)*Q(r)==P(r)*F(r)}")

evaluaciones_S_en_r = evaluar_polinomios_en_punto(construir_polinomios_de_partes(shares_S, n_parts, X), r)
evaluaciones_Q_en_r = evaluar_polinomios_en_punto(construir_polinomios_de_partes(shares_Q, n_parts, X), r)
evaluaciones_P_en_r = evaluar_polinomios_en_punto(construir_polinomios_de_partes(shares_P, n_parts, X), r)

# Paso 3: Calcular [alpha] y [beta] para cada parte
def calcular_alpha_beta(epsilon, evaluaciones_Q, evaluaciones_S, a_shares, b_shares):
    comparticion_alpha = [epsilon * evaluaciones_Q[j] + a_shares[j] for j in range(n_parts)]
    comparticion_beta = [evaluaciones_S[j] + b_shares[j] for j in range(n_parts)]
    return comparticion_alpha, comparticion_beta

comparticion_alpha, comparticion_beta = calcular_alpha_beta(epsilon, evaluaciones_Q_en_r, evaluaciones_S_en_r, a_shares, b_shares)

# Paso 4: Transmitir [alpha] y [beta] para obtener alpha y beta
def reconstruir_valor(comparticiones):
    return sum(comparticiones)

alpha = reconstruir_valor(comparticion_alpha)
beta = reconstruir_valor(comparticion_beta)
print(f"\nValor de alpha: {alpha}")
print(f"Valor de beta: {beta}")

# Paso 4: Calcular [v] para cada parte
# Calcular c como la multiplicación de la suma de a_shares y la suma de b_shares, luego dividir c en partes aleatorias
def calcular_v(epsilon, evaluaciones_P, a_shares, b_shares, alpha, beta, c_shares):
    comparticion_v = []
    for j in range(n_parts):
        if j == 0:
            # Para el primer elemento, restamos alpha * beta
            v_j = (epsilon * F(r) * evaluaciones_P[j]) - c_shares[j] + alpha * b_shares[j] + beta * a_shares[j] - alpha * beta
        else:
            # Para los demás elementos, no restamos alpha * beta
            v_j = (epsilon * F(r) * evaluaciones_P[j]) - c_shares[j] + alpha * b_shares[j] + beta * a_shares[j]
        comparticion_v.append(v_j)
    return comparticion_v

comparticion_v = calcular_v(epsilon, evaluaciones_P_en_r, a_shares, b_shares, alpha, beta, c_shares)

# Paso 5: Transmitir [v] para obtener v
v = reconstruir_valor(comparticion_v)
print(f"\nValor de v: {v}")

# Paso 7: Verificar si v es igual a 0
def verificar_v(v):
    if v == 0:
        print("Las partes aceptan: v = 0")
    else:
        print(f"Las partes rechazan: v = {v}")

verificar_v(v)

Valor aleatorio 22
Valuación en punto aleatorio True

Valor de alpha: 0
Valor de beta: 16

Valor de v: 0
Las partes aceptan: v = 0


In [8]:
# Paso 4: Derivación del segundo desafío y selección del subconjunto de partes a abrir

# Concatenar h1 y los valores de difusión alpha, beta, v para formar la entrada de la función hash
valores_intermedios_concatenados = (
    mensaje+
    str(h1_int) + 
    ''.join(str(alpha_val) for alpha_val in comparticion_alpha) +
    ''.join(str(beta_val) for beta_val in comparticion_beta) +
    ''.join(str(v_val) for v_val in comparticion_v)
)

# Calcular h2 aplicando SHA-256 a los valores intermedios concatenados
h2_hex = hashlib.sha256(valores_intermedios_concatenados.encode()).hexdigest()
print(f"\nValor de h2 (hash de los valores intermedios concatenados): {h2_hex}")

# Convertir h2 de hexadecimal a un valor numérico para usarlo como semilla
h2_int = int(h2_hex, 16)

# Inicializar el generador aleatorio con la semilla h2
random.seed(h2_int)

# Determinar el subconjunto de partes I a abrir
# Vamos a seleccionar aleatoriamente la mitad de las partes para abrir
numero_partes_a_abrir = n_parts // 2
indices_revelados = random.sample(range(n_parts), numero_partes_a_abrir)

print(f"\nÍndices de las partes que se deben abrir: {indices_revelados}")


Valor de h2 (hash de los valores intermedios concatenados): 743086438e8a260b13b483e614dbb773333f7d872735db2f53e5dd15582c9500

Índices de las partes que se deben abrir: [1, 3]


In [9]:
# Paso 5: Revelación de las partes seleccionadas en el conjunto I

# Crear una lista ordenada para almacenar los valores de todas las partes
partes = []

print("\nRevelando las comparticiones para los índices seleccionados:")

# Extraer los valores correspondientes para cada índice y guardar en la lista 'partes'
for idx in range(n_parts):
    # Determinar si la parte está en el conjunto de las reveladas (I)
    es_revelada = idx in indices_revelados

    # Si la parte está revelada, guardamos todos los valores
    if es_revelada:
        parte = {
            "indice": idx,
            "es_revelada": True,
            "coef_S": shares_S[idx],
            "coef_Q": shares_Q[idx],
            "coef_P": shares_P[idx],
            "parte_a": a_shares[idx],
            "parte_b": b_shares[idx],
            "parte_c": c_shares[idx],
            "compromiso": commits[idx],
            "valor_aleatorio": valores_aleatorios[idx],
            "alpha": comparticion_alpha[idx],
            "beta": comparticion_beta[idx],
            "v": comparticion_v[idx],
        }

        partes.append(parte)

        # Imprimir información de la parte revelada
        print(f"\nParte {idx + 1} revelada:")
        for clave, valor in parte.items():
            if clave not in ["indice", "es_revelada"]:  # No imprimir el índice ni la bandera
                print(f"  - {clave}: {valor}")

    # Si la parte no está revelada, solo guardamos los valores necesarios para la verificación
    else:
        parte = {
            "indice": idx,
            "es_revelada": False,
            "alpha": comparticion_alpha[idx],
            "beta": comparticion_beta[idx],
            "v": comparticion_v[idx],
            "compromiso": commits[idx],
        }

        partes.append(parte)

        # Imprimir información de la parte no revelada
        print(f"\nParte {idx + 1} no revelada:")
        for clave, valor in parte.items():
            if clave not in ["indice", "es_revelada"]:  # No imprimir el índice ni la bandera
                print(f"  - {clave}: {valor}")
                
print(f"\nValor de h_1: \n{h1_hex}")
print(f"\nValor de h_2: \n{h2_hex}")


Revelando las comparticiones para los índices seleccionados:

Parte 1 no revelada:
  - alpha: 22
  - beta: 21
  - v: 12
  - compromiso: fe5aee9ed25f6ecaf1d1275d59a77904bdd9fade7c66eefbc5b62cd502e9e1f6

Parte 2 revelada:
  - coef_S: [17, 22, 9, 10, 13, 21]
  - coef_Q: [20, 5, 16]
  - coef_P: [6, 1]
  - parte_a: 16
  - parte_b: 9
  - parte_c: 0
  - compromiso: 120f694616028fe9c6966faf3168ee9120a22c800d01162fb673478f01ae0014
  - valor_aleatorio: 4
  - alpha: 21
  - beta: 18
  - v: 22

Parte 3 no revelada:
  - alpha: 13
  - beta: 16
  - v: 11
  - compromiso: 12ab9e4054b4562477a9c593f64a1dc1ddaa9ef897e469436f30f5597258bad4

Parte 4 revelada:
  - coef_S: [12, 21, 6, 20, 21, 1]
  - coef_Q: [5, 2, 8]
  - coef_P: [19, 14]
  - parte_a: 2
  - parte_b: 12
  - parte_c: 18
  - compromiso: 7977ac33042c57e71e631eaad7ad6b653a7950501d82e66c31f19f41cd6667fa
  - valor_aleatorio: 21
  - alpha: 6
  - beta: 9
  - v: 10

Parte 5 no revelada:
  - alpha: 7
  - beta: 21
  - v: 14
  - compromiso: 65315fffdc8313e

In [10]:
firma = [h1_hex, h2_hex, partes]
firma

['f8b4c73af60f109f90bea2428462694e62cf37debb3db80a127bb7c95d34a7d5',
 '743086438e8a260b13b483e614dbb773333f7d872735db2f53e5dd15582c9500',
 [{'indice': 0,
   'es_revelada': False,
   'alpha': 22,
   'beta': 21,
   'v': 12,
   'compromiso': 'fe5aee9ed25f6ecaf1d1275d59a77904bdd9fade7c66eefbc5b62cd502e9e1f6'},
  {'indice': 1,
   'es_revelada': True,
   'coef_S': [17, 22, 9, 10, 13, 21],
   'coef_Q': [20, 5, 16],
   'coef_P': [6, 1],
   'parte_a': 16,
   'parte_b': 9,
   'parte_c': 0,
   'compromiso': '120f694616028fe9c6966faf3168ee9120a22c800d01162fb673478f01ae0014',
   'valor_aleatorio': 4,
   'alpha': 21,
   'beta': 18,
   'v': 22},
  {'indice': 2,
   'es_revelada': False,
   'alpha': 13,
   'beta': 16,
   'v': 11,
   'compromiso': '12ab9e4054b4562477a9c593f64a1dc1ddaa9ef897e469436f30f5597258bad4'},
  {'indice': 3,
   'es_revelada': True,
   'coef_S': [12, 21, 6, 20, 21, 1],
   'coef_Q': [5, 2, 8],
   'coef_P': [19, 14],
   'parte_a': 2,
   'parte_b': 12,
   'parte_c': 18,
   'compromiso

## Verificación

In [21]:
mensaje_firma = "Este es el mensaje a firmar."

In [22]:
(h1_firma, h2_firma, partes_firma) = firma

print(f"h1 de firma: {h1_firma}")
print(f"h2 de firma: {h2_firma}")
print(f"partes: {partes})")

h1 de firma: f8b4c73af60f109f90bea2428462694e62cf37debb3db80a127bb7c95d34a7d5
h2 de firma: 743086438e8a260b13b483e614dbb773333f7d872735db2f53e5dd15582c9500
partes: [{'indice': 0, 'es_revelada': False, 'alpha': 22, 'beta': 21, 'v': 12, 'compromiso': 'fe5aee9ed25f6ecaf1d1275d59a77904bdd9fade7c66eefbc5b62cd502e9e1f6'}, {'indice': 1, 'es_revelada': True, 'coef_S': [17, 22, 9, 10, 13, 21], 'coef_Q': [20, 5, 16], 'coef_P': [6, 1], 'parte_a': 16, 'parte_b': 9, 'parte_c': 0, 'compromiso': '120f694616028fe9c6966faf3168ee9120a22c800d01162fb673478f01ae0014', 'valor_aleatorio': 4, 'alpha': 21, 'beta': 18, 'v': 22}, {'indice': 2, 'es_revelada': False, 'alpha': 13, 'beta': 16, 'v': 11, 'compromiso': '12ab9e4054b4562477a9c593f64a1dc1ddaa9ef897e469436f30f5597258bad4'}, {'indice': 3, 'es_revelada': True, 'coef_S': [12, 21, 6, 20, 21, 1], 'coef_Q': [5, 2, 8], 'coef_P': [19, 14], 'parte_a': 2, 'parte_b': 12, 'parte_c': 18, 'compromiso': '7977ac33042c57e71e631eaad7ad6b653a7950501d82e66c31f19f41cd6667fa', 

In [23]:
# Paso 1: Recalcular los compromisos de las partes reveladas

print("\nVerificando los compromisos de las partes reveladas...")

compromisos_validos = True

# Iterar sobre todas las partes almacenadas en la lista 'partes'
for parte in partes_firma:
    # Solo verificamos las partes reveladas
    if parte["es_revelada"]:
        # Obtener los valores de las entradas reveladas
        coef_S = parte["coef_S"]
        coef_P = parte["coef_P"]
        coef_Q = parte["coef_Q"]
        parte_a = parte["parte_a"]
        parte_b = parte["parte_b"]
        parte_c = parte["parte_c"]
        valor_aleatorio = parte["valor_aleatorio"]

        # Recalcular el compromiso usando una función hash
        entrada_compromiso = f"{coef_S}{coef_Q}{coef_P}{parte_a}{parte_b}{parte_c}{valor_aleatorio}"
        compromiso_recalculado = hashlib.sha256(entrada_compromiso.encode()).hexdigest()

        # Obtener el compromiso original
        compromiso_original = parte["compromiso"]

        # Comparar el compromiso recalculado con el compromiso original
        if compromiso_recalculado != compromiso_original:
            print(f"Error: El compromiso recalculado no coincide para la parte {parte['indice'] + 1}.")
            compromisos_validos = False
        else:
            print(f"El compromiso recalculado coincide para la parte {parte['indice'] + 1}.")

# Determinar si todos los compromisos fueron verificados correctamente
if compromisos_validos:
    print("\nTodos los compromisos de las partes reveladas son válidos.")
else:
    print("\nAlgunos compromisos de las partes reveladas no son válidos. Verificación fallida.")


Verificando los compromisos de las partes reveladas...
El compromiso recalculado coincide para la parte 2.
El compromiso recalculado coincide para la parte 4.

Todos los compromisos de las partes reveladas son válidos.


In [24]:
# Paso 2: Derivar el primer desafío h1 durante la verificación (nueva derivación con nombre diferente)

# Concatenar todos los compromisos de las partes (en el orden original) para formar una única cadena
compromisos_concatenados = mensaje_firma + ''

# Iterar sobre todas las partes para concatenar los compromisos en orden
for parte in partes:    
    compromisos_concatenados += parte["compromiso"]

# Calcular h1_verificacion aplicando SHA-256 a la cadena de compromisos concatenados
h1_verificacion_hex = hashlib.sha256(compromisos_concatenados.encode()).hexdigest()
print(f"\nValor de h1_verificacion (hash de los compromisos concatenados): {h1_verificacion_hex}")

# Convertir h1_verificacion de hexadecimal a un valor numérico para usarlo como semilla
h1_verificacion_int = int(h1_verificacion_hex, 16)

# Inicializar el generador aleatorio con la semilla h1_verificacion
random.seed(h1_verificacion_int)

# Derivar r y epsilon a partir del generador aleatorio
F_p_int = int(F_p.order())  # Obtiene el orden del campo finito como un entero
r = random.randint(1, F_p_int - 1)  # Generar un valor aleatorio r en el campo finito F_p
epsilon = random.randint(1, F_p_int - 1)  # Generar un valor aleatorio epsilon en el campo finito F_p

# Convertir r y epsilon a elementos del campo finito F_p
r = F_p(r)
epsilon = F_p(epsilon)

print(f"\nValores derivados de h1_verificacion durante la verificación:")
print(f"r: {r}")
print(f"epsilon: {epsilon}")


Valor de h1_verificacion (hash de los compromisos concatenados): 3b0f1a6176f70616f9f635b72de2b0cc11bd7f8ead7f28b0f1b71e52c5e267da

Valores derivados de h1_verificacion durante la verificación:
r: 12
epsilon: 2


In [25]:
# Paso 3: Calcular [alpha] y [beta] para cada parte revelada y verificar si coinciden

print("\nCalculando y verificando [alpha] y [beta] para las partes reveladas...")

# Lista para almacenar el resultado de la verificación de cada parte
resultados_verificacion = []

# Iterar sobre todas las partes en la lista 'partes'
for parte in partes_firma:
    # Solo procesar las partes reveladas
    if parte["es_revelada"]:
        # Obtener los valores necesarios
        coef_S = parte["coef_S"]
        coef_Q = parte["coef_Q"]
        coef_P = parte["coef_P"]
        parte_a = parte["parte_a"]
        parte_b = parte["parte_b"]
        valor_aleatorio = parte["valor_aleatorio"]

        # Calcular la evaluación del polinomio S y Q en el punto r
        evaluacion_S = sum(coef * (r ** i) for i, coef in enumerate(coef_S))
        evaluacion_Q = sum(coef * (r ** i) for i, coef in enumerate(coef_Q))

        # Calcular los valores de alpha y beta
        alpha_recalculado = epsilon * evaluacion_Q + parte_a
        beta_recalculado = evaluacion_S + parte_b

        # Obtener los valores originales para comparar
        alpha_original = parte["alpha"]
        beta_original = parte["beta"]

        # Verificar si los valores recalculados coinciden con los originales
        alpha_valido = (alpha_recalculado == alpha_original)
        beta_valido = (beta_recalculado == beta_original)

        # Guardar el resultado de la verificación
        resultados_verificacion.append((parte["indice"], alpha_valido, beta_valido))

        # Imprimir el resultado de la verificación para esta parte
        print(f"\nParte {parte['indice'] + 1}:")
        print(f"  Alpha recalculado: {alpha_recalculado}, Alpha original: {alpha_original}")
        print("  Alpha - Comparación: " + ("Iguales" if alpha_valido else "Diferentes"))
        print(f"  Beta recalculado: {beta_recalculado}, Beta original: {beta_original}")
        print("  Beta - Comparación: " + ("Iguales" if beta_valido else "Diferentes"))

# Determinar si todos los valores de [alpha] y [beta] fueron verificados correctamente
alpha_beta_validos = all(alpha_valido and beta_valido for _, alpha_valido, beta_valido in resultados_verificacion)

if alpha_beta_validos:
    print("\nTodos los valores de [alpha] y [beta] para las partes reveladas son válidos.")
else:
    print("\nAlgunos valores de [alpha] y [beta] para las partes reveladas no son válidos. Verificación fallida.")


Calculando y verificando [alpha] y [beta] para las partes reveladas...

Parte 2:
  Alpha recalculado: 0, Alpha original: 21
  Alpha - Comparación: Diferentes
  Beta recalculado: 1, Beta original: 18
  Beta - Comparación: Diferentes

Parte 4:
  Alpha recalculado: 18, Alpha original: 6
  Alpha - Comparación: Diferentes
  Beta recalculado: 19, Beta original: 9
  Beta - Comparación: Diferentes

Algunos valores de [alpha] y [beta] para las partes reveladas no son válidos. Verificación fallida.


In [26]:
# Paso 4: Derivar el segundo desafío h2_verificacion durante la verificación

# Concatenar h1_verificacion y los valores de difusión (alpha, beta, v) para formar la entrada de la función hash
valores_intermedios_concatenados = (
    mensaje_firma +
    str(h1_verificacion_int) +  # Concatenar el valor de h1_verificacion como un entero
    ''.join(str(parte["alpha"]) for parte in partes) +  # Concatenar todos los valores de alpha
    ''.join(str(parte["beta"]) for parte in partes) +  # Concatenar todos los valores de beta
    ''.join(str(parte["v"]) for parte in partes)  # Concatenar todos los valores de v
)

# Calcular h2_verificacion aplicando SHA-256 a la cadena de valores intermedios concatenados
h2_verificacion_hex = hashlib.sha256(valores_intermedios_concatenados.encode()).hexdigest()
print(f"\nValor de h2_verificacion (hash de los valores intermedios concatenados): {h2_verificacion_hex}")

# Convertir h2_verificacion de hexadecimal a un valor numérico para usarlo como semilla
h2_verificacion_int = int(h2_verificacion_hex, 16)


Valor de h2_verificacion (hash de los valores intermedios concatenados): 5fa70989f89eda6a74b00fbb866c97b578cbe1e065c1a75fbc64ed8727587317


In [27]:
# Paso final: Verificación y Decisión de Aceptación o Rechazo de la Prueba de Cero Conocimiento

print("\nVerificando las partes seleccionadas en el subconjunto I...")

# Comparar los valores de h1 y h2 originales con los recalculados durante la verificación
h1_valido = (h1_verificacion_hex == h1_firma)
h2_valido = (h2_verificacion_hex == h2_firma)

# Imprimir los resultados de las comparaciones de h1 y h2
print("\nComparación de valores hash:")
print(f"  Valor original de h1: {h1_firma}, Valor verificado de h1: {h1_verificacion_hex}")
print("  h1 - Comparación: " + ("Iguales" if h1_valido else "Diferentes"))

print(f"  Valor original de h2: {h2_firma}, Valor verificado de h2: {h2_verificacion_hex}")
print("  h2 - Comparación: " + ("Iguales" if h2_valido else "Diferentes"))

# Verificar si todos los compromisos son válidos para las partes seleccionadas
if compromisos_validos and alpha_beta_validos and h1_valido and h2_valido:
    print("\nVerificación exitosa: La prueba de cero conocimiento ha sido aceptada.")
else:
    print("\nVerificación fallida: La prueba de cero conocimiento ha sido rechazada.")


Verificando las partes seleccionadas en el subconjunto I...

Comparación de valores hash:
  Valor original de h1: f8b4c73af60f109f90bea2428462694e62cf37debb3db80a127bb7c95d34a7d5, Valor verificado de h1: 3b0f1a6176f70616f9f635b72de2b0cc11bd7f8ead7f28b0f1b71e52c5e267da
  h1 - Comparación: Diferentes
  Valor original de h2: 743086438e8a260b13b483e614dbb773333f7d872735db2f53e5dd15582c9500, Valor verificado de h2: 5fa70989f89eda6a74b00fbb866c97b578cbe1e065c1a75fbc64ed8727587317
  h2 - Comparación: Diferentes

Verificación fallida: La prueba de cero conocimiento ha sido rechazada.
